## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-22-01-05-38 +0000,bbc,Why Croatia's capital wants to hold Europe's b...,https://www.bbc.com/news/articles/c8dy5m56477o...
1,2025-12-22-01-05-30 +0000,nypost,Cartoon PSA advises to flush rats if climbing ...,https://nypost.com/2025/12/21/us-news/cartoon-...
2,2025-12-22-01-00-00 +0000,wsj,These Teenagers Are Already Running Their Own ...,https://www.wsj.com/business/entrepreneurship/...
3,2025-12-22-01-00-00 +0000,wsj,Oil Rises Amid Growing U.S.-Venezuela Tensions,https://www.wsj.com/finance/commodities-future...
4,2025-12-22-00-58-09 +0000,nyt,Waymo Suspended Service in San Francisco After...,https://www.nytimes.com/2025/12/21/us/waymo-su...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
272,trump,15
106,epstein,11
28,oil,10
46,bondi,9
32,venezuela,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
181,2025-12-21-01-59-17 +0000,wapo,Coast Guard boards oil tanker leaving Venezuel...,https://www.washingtonpost.com/national-securi...,67
69,2025-12-21-20-20-00 +0000,wsj,"Before Charlie Kirk was fatally shot, he was r...",https://www.wsj.com/politics/policy/charlie-ki...,56
96,2025-12-21-17-30-34 +0000,nypost,US military targeting third tanker off coast o...,https://nypost.com/2025/12/21/world-news/us-mi...,56
74,2025-12-21-19-56-00 +0000,wsj,A top Justice Department official said Sunday ...,https://www.wsj.com/politics/policy/justice-de...,52
84,2025-12-21-18-40-44 +0000,nyt,Epstein Files Photos Disappear From Government...,https://www.nytimes.com/2025/12/20/us/politics...,45


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
181,67,2025-12-21-01-59-17 +0000,wapo,Coast Guard boards oil tanker leaving Venezuel...,https://www.washingtonpost.com/national-securi...
69,56,2025-12-21-20-20-00 +0000,wsj,"Before Charlie Kirk was fatally shot, he was r...",https://www.wsj.com/politics/policy/charlie-ki...
74,52,2025-12-21-19-56-00 +0000,wsj,A top Justice Department official said Sunday ...,https://www.wsj.com/politics/policy/justice-de...
4,39,2025-12-22-00-58-09 +0000,nyt,Waymo Suspended Service in San Francisco After...,https://www.nytimes.com/2025/12/21/us/waymo-su...
125,35,2025-12-21-11-01-08 +0000,nypost,Defiant sister of slain Bondi Beach rabbi — wh...,https://nypost.com/2025/12/21/world-news/defia...
48,31,2025-12-21-21-54-22 +0000,nypost,New Yorkers slam new anti-fare-beating turnsti...,https://nypost.com/2025/12/21/us-news/mta-inst...
56,31,2025-12-21-21-19-00 +0000,nypost,Cops raid 2 more NYC knock-off markets — scari...,https://nypost.com/2025/12/21/us-news/cops-rai...
150,29,2025-12-21-09-52-21 +0000,nypost,"Australian PM Albanese booed, heckled as he si...",https://nypost.com/2025/12/21/world-news/austr...
26,26,2025-12-21-23-28-58 +0000,nypost,South Carolina town committee keeps Nativity s...,https://nypost.com/2025/12/21/us-news/south-ca...
132,26,2025-12-21-11-00-00 +0000,wsj,Cleveland Fed President Beth Hammack said she ...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
